In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
MET_forecast = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/met_forecast.parquet")
MET_nowcast = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/met_nowcast.parquet")
Power_data = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/wind_power_per_bidzone.parquet")
Meta_data = pd.read_csv("/home2/s5549329/windAI_rug/WindAi/given_datasets/windparks_bidzone.csv")

In [3]:
Power_data_long = Power_data.reset_index().melt(
    id_vars="index",            
    var_name="bidding_area",     
    value_name="power_MW"        
).rename(columns={"index": "time"})

Power_data_long.head(5)

,time,bidding_area,power_MW
0,2020-01-01 00:00:00,ELSPOT NO1,149.285262
1,2020-01-01 01:00:00,ELSPOT NO1,152.634024
2,2020-01-01 02:00:00,ELSPOT NO1,151.163256
3,2020-01-01 03:00:00,ELSPOT NO1,150.223341
4,2020-01-01 04:00:00,ELSPOT NO1,157.415142


Data aligned with the same time range

In [4]:
start = max(
    MET_forecast['time'].min(),
    MET_nowcast.index.min(),
    Power_data_long['time'].min()
)

end = min(
    MET_forecast['time'].max(),
    MET_nowcast.index.max(),
    Power_data_long['time'].max()
)

In [5]:
MET_forecast = MET_forecast[(MET_forecast['time'] >= start) & (MET_forecast['time'] <= end)]
MET_nowcast = MET_nowcast[(MET_nowcast.index >= start) & (MET_nowcast.index <= end)]
Power_data_long = Power_data_long[(Power_data_long['time'] >= start) & (Power_data_long['time'] <= end)]

In [6]:
MET_forecast.head(5)

,sid,time_ref,time,lt,ws10m_00,ws10m_01,ws10m_02,ws10m_03,ws10m_04,ws10m_05,...,g10m_05,g10m_06,g10m_07,g10m_08,g10m_09,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14
0,Engerfjellet,2020-02-15 12:00:00,2020-02-15 12:00:00,0,2.096957,2.110652,2.304629,1.942044,1.447011,2.288181,...,6.918935,8.110546,5.988888,6.934274,5.235212,5.507061,7.193576,6.753938,4.841811,4.875890
1,Engerfjellet,2020-02-15 12:00:00,2020-02-15 13:00:00,1,3.449402,3.436193,3.205872,3.450461,3.143307,4.278822,...,9.914339,9.184751,6.999992,7.892150,7.214608,8.252510,9.535547,8.717684,8.152432,6.240438
2,Engerfjellet,2020-02-15 12:00:00,2020-02-15 14:00:00,2,3.348926,2.903557,2.779971,3.733802,3.148597,2.912592,...,10.031385,8.624619,7.984353,8.264300,7.205638,9.052546,9.531441,8.367238,7.840395,7.153080
3,Engerfjellet,2020-02-15 12:00:00,2020-02-15 15:00:00,3,3.095971,2.560161,2.448126,2.267286,3.050008,3.310141,...,8.185250,8.821701,7.481009,8.172009,6.255360,7.345034,8.303812,6.735660,7.600200,7.189164
4,Engerfjellet,2020-02-15 12:00:00,2020-02-15 16:00:00,4,2.821545,2.965743,3.235484,3.212730,3.064931,2.420390,...,7.551644,9.709259,8.365305,8.703517,6.151643,10.116440,7.346225,9.079601,9.048587,6.937930


In [7]:
len(MET_forecast["sid"].unique())

61

In [8]:
MET_nowcast.head(5)

,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,windpark
time,,,,,,,
2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet
2020-02-15 13:00:00,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,Bessakerfjellet
2020-02-15 14:00:00,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,Bessakerfjellet
2020-02-15 15:00:00,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,Bessakerfjellet
2020-02-15 16:00:00,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,Bessakerfjellet


In [9]:
len(MET_nowcast["windpark"].unique())

61

In [10]:
Meta_data.head(5)

,bidding_area,substation_name,operating_power_max,prod_start_new,eic_code
0,ELSPOT NO1,Engerfjellet,52.8,2022-09-19 13:00:00,50WP00000002158R
1,ELSPOT NO1,Hån Vindpark,21.0,2022-10-26 22:00:00,50WP000000022251
2,ELSPOT NO1,Kjølberget,55.9,2020-07-07 09:00:00,50WP00000002085S
3,ELSPOT NO1,Marker Vindpark,54.0,2020-01-01 00:00:00,50WP00000002048Y
4,ELSPOT NO1,Raskiftet,111.6,2020-01-01 00:00:00,50WP00000001718K


In [11]:
len(Meta_data["substation_name"].unique())

65

Data alligned with the same windparks

In [12]:
meta_set = set(Meta_data["substation_name"])
nowcast_set = set(MET_nowcast["windpark"])
forecast_set = set(MET_forecast["sid"])

common = meta_set & nowcast_set & forecast_set
print(f"Common windparks in all datasets: {len(common)}")
print(sorted(common))

to_drop_from_meta = meta_set - common
to_drop_from_nowcast = nowcast_set - common
to_drop_from_forecast = forecast_set - common

print("\n To drop from Meta_data:")
print(sorted(to_drop_from_meta))

print("\n To drop from MET_nowcast:")
print(sorted(to_drop_from_nowcast))

print("\n To drop from MET_forecast:")
print(sorted(to_drop_from_forecast))

Common windparks in all datasets: 59
['Bessakerfjellet', 'Bjerk_VK Vindpark', 'Buheii Vindpark', 'Dønnesfjord Vind', 'Egersund Vindkrv', 'Einarsdalen', 'Engerfjellet', 'Fakken', 'Frøya Vindpark', 'Geitfjellet', 'Gismarvik Vindpark', 'Guleslettene Vindpark', 'Hamnefjell', 'Haraheia', 'Haram Kraft', 'Harbaksfjellet', 'Havøygavlen', 'Hennøy', 'Hitra', 'Hundhammerfjelle', 'Hån Vindpark', 'Høg Jæren', 'Kjølberget', 'Kjøllefjord vindpark', 'Kvenndalsfjellet', 'Kvitfjell vindpark', 'Lista VK', 'Lutelandet', 'Marker Vindpark', 'Mehuken', 'Midtfjellet', 'Måkaknuten', 'Nygårdsfjellet', 'Okla Vindkraftverk', 'Raggovidda', 'Raskiftet', 'Raudfjell Vindpark', 'Sandøy Vindkraft', 'Skomakerfjellet', 'Skudeneshavn', 'Smøla', 'Songkjølen', 'Stokkeland', 'Stokkfjellet', 'Storheia', 'Storøy vindpark', 'Svåheia', 'Sørmarkfjellet', 'Tellenes', 'Tindafjellet', 'Tysvær Vindpark', 'Valsn_Vimle', 'Valsneset', 'Vardafjell', 'Ytre Vikna', 'Ånstadblåheia', 'Øie', 'Øyfjell1', 'Øyfjell2']

 To drop from Meta_data:
[

In [13]:
Meta_data_aligned = Meta_data[Meta_data["substation_name"].isin(common)]
MET_nowcast_aligned = MET_nowcast[MET_nowcast["windpark"].isin(common)]
MET_forecast_aligned = MET_forecast[MET_forecast["sid"].isin(common)]

In [14]:
MET_forecast_aligned = MET_forecast_aligned.rename(columns={"sid": "windpark"})

if MET_nowcast_aligned.index.name == 'time':
    MET_nowcast_aligned = MET_nowcast_aligned.reset_index()
if MET_forecast_aligned.index.name == 'time':
    MET_forecast_aligned = MET_forecast_aligned.reset_index()

weather_combined = pd.merge(
    MET_nowcast_aligned,
    MET_forecast_aligned,
    on=["windpark", "time"],
    how="inner",
    suffixes=("_nowcast", "_forecast")
)

full_data = pd.merge(
    weather_combined,
    Meta_data_aligned,
    left_on="windpark",
    right_on="substation_name",
    how="left"
)

full_data.head(5)

,time,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,windpark,time_ref,lt,...,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14,bidding_area,substation_name,operating_power_max,prod_start_new,eic_code
0,2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet,2020-02-15 12:00:00,0,...,17.925253,16.365322,14.280454,12.755079,13.875360,ELSPOT NO3,Bessakerfjellet,57.5,2020-01-01 00:00:00,50WP00000000044F
1,2020-02-15 13:00:00,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,Bessakerfjellet,2020-02-15 12:00:00,1,...,15.573839,14.297405,12.121831,15.328096,10.223662,ELSPOT NO3,Bessakerfjellet,57.5,2020-01-01 00:00:00,50WP00000000044F
2,2020-02-15 14:00:00,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,Bessakerfjellet,2020-02-15 12:00:00,2,...,12.315957,12.458931,12.071150,11.322252,10.631420,ELSPOT NO3,Bessakerfjellet,57.5,2020-01-01 00:00:00,50WP00000000044F
3,2020-02-15 15:00:00,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,Bessakerfjellet,2020-02-15 12:00:00,3,...,8.903479,8.134869,8.383938,7.247891,8.257983,ELSPOT NO3,Bessakerfjellet,57.5,2020-01-01 00:00:00,50WP00000000044F
4,2020-02-15 16:00:00,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,Bessakerfjellet,2020-02-15 12:00:00,4,...,6.814418,7.093896,7.630539,8.442563,6.639053,ELSPOT NO3,Bessakerfjellet,57.5,2020-01-01 00:00:00,50WP00000000044F


In [15]:
full_data.drop(columns=['eic_code', 'substation_name', 'prod_start_new', 'time_ref', 'lt', 'operating_power_max'], inplace=True)
full_data.head(5)

,time,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,windpark,ws10m_00,ws10m_01,...,g10m_06,g10m_07,g10m_08,g10m_09,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14,bidding_area
0,2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet,7.382969,9.197692,...,14.044403,14.188225,13.297676,13.375314,17.925253,16.365322,14.280454,12.755079,13.875360,ELSPOT NO3
1,2020-02-15 13:00:00,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,Bessakerfjellet,6.468776,7.247953,...,12.194210,11.443217,10.773432,13.224432,15.573839,14.297405,12.121831,15.328096,10.223662,ELSPOT NO3
2,2020-02-15 14:00:00,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,Bessakerfjellet,5.207288,6.332892,...,8.152608,14.249521,10.841598,10.941404,12.315957,12.458931,12.071150,11.322252,10.631420,ELSPOT NO3
3,2020-02-15 15:00:00,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,Bessakerfjellet,3.396515,3.428589,...,7.233331,7.699943,8.996366,7.405312,8.903479,8.134869,8.383938,7.247891,8.257983,ELSPOT NO3
4,2020-02-15 16:00:00,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,Bessakerfjellet,4.159583,4.186426,...,7.801557,8.343901,6.963749,7.973725,6.814418,7.093896,7.630539,8.442563,6.639053,ELSPOT NO3


In [16]:
Power_data_long.head(5)

,time,bidding_area,power_MW
1056,2020-02-15 12:00:00,ELSPOT NO1,72.052233
1057,2020-02-15 13:00:00,ELSPOT NO1,44.666173
1058,2020-02-15 14:00:00,ELSPOT NO1,65.059451
1059,2020-02-15 15:00:00,ELSPOT NO1,86.562093
1060,2020-02-15 16:00:00,ELSPOT NO1,77.712180


In [ ]:
df= pd.merge(
    full_data,
    Power_data_long,
    on=['time', 'bidding_area'],
    how='inner'  
)
df.head(5)

,time,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,windpark,ws10m_00,ws10m_01,...,g10m_07,g10m_08,g10m_09,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14,bidding_area,power_MW
0,2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet,7.382969,9.197692,...,14.188225,13.297676,13.375314,17.925253,16.365322,14.280454,12.755079,13.875360,ELSPOT NO3,588.636703
1,2020-02-15 12:00:00,278.131165,99809.203125,0.903388,6.924076e-03,6.928626,241.437683,Einarsdalen,6.776339,9.097810,...,13.328640,13.357601,14.944031,18.085197,16.130472,13.793073,13.672788,12.879631,ELSPOT NO3,588.636703
2,2020-02-15 12:00:00,278.971771,99818.320312,0.802680,2.932219e-09,7.764953,226.100189,Frøya Vindpark,7.558401,7.949138,...,16.473120,15.874034,14.517097,14.946919,13.509794,15.290257,13.769015,14.388791,ELSPOT NO3,588.636703
3,2020-02-15 12:00:00,274.326752,99916.242188,0.788108,3.593363e-09,4.852487,245.355621,Geitfjellet,4.280652,4.500379,...,11.637486,12.888701,12.246942,11.493663,11.278106,11.106653,9.085333,12.564517,ELSPOT NO3,588.636703
4,2020-02-15 12:00:00,274.859680,99822.750000,0.884574,6.714873e-04,10.735115,161.846939,Guleslettene Vindpark,10.608345,10.306207,...,15.320757,16.165518,15.248836,14.163930,15.367685,16.001540,15.497243,16.134457,ELSPOT NO3,588.636703


: 

In [ ]:
region_park_counts = (
    full_data[['bidding_area', 'windpark']]
    .drop_duplicates()
    .groupby('bidding_area')
    .size()
    .reset_index(name='num_windparks')
)

full_data_no_park = full_data.drop(columns=['windpark'])

df_region = pd.merge(
    full_data_no_park,
    Power_data_long,
    on=['time', 'bidding_area'],
    how='inner'
)

df_region = pd.merge(
    df_region,
    region_park_counts,
    on='bidding_area',
    how='left'
)